In [17]:
import pandas as pd 
from typing import Dict, Optional, List

In [14]:
lotto = pd.read_csv("./output/merged_lotto.csv")
lotto_plus = pd.read_csv("./output/merged_lottoplus.csv")
mini_lotto = pd.read_csv("./output/mini_lotto.csv")

print(lotto)

        number        date            numbers  szostka_count  szostka_value  \
0           47  22-12-1957   8,15,24,27,37,46              0            0.0   
1           46  15-12-1957   7,12,19,36,42,49              0            0.0   
2           45  08-12-1957  11,17,24,35,42,47              0            0.0   
3           44  01-12-1957    5,8,14,26,32,44              0            0.0   
4           43  24-11-1957    2,6,18,20,40,49              0            0.0   
...        ...         ...                ...            ...            ...   
201003    6986  11-01-2024   2,13,17,26,34,38              0            0.0   
201004    6985  09-01-2024   4,19,20,23,28,48              0            0.0   
201005    6984  06-01-2024      2,5,8,9,30,42              0            0.0   
201006    6983  04-01-2024  10,25,26,28,44,46              0            0.0   
201007    6982  02-01-2024  15,18,28,30,37,49              0            0.0   

        piatka_count  piatka_value  czworka_count  

In [18]:
def get_all_numbers(dataset) -> List:
    all_numbers = []
    for numbers_str in dataset['numbers']:
        numbers_list = numbers_str.split(',')
        all_numbers.extend(map(int, numbers_list))
    
    return all_numbers

In [19]:


def get_most_common_numbers(dataset, num_results: int = 6) -> Dict[Optional[int], Optional[int]]:
    all_numbers = get_all_numbers(dataset)
    
    counts_df = pd.DataFrame(all_numbers, columns=['number']).groupby('number').size().reset_index(name='count')
    
    counts_df = counts_df.sort_values(by='count', ascending=False)
    
    return counts_df.head(num_results)


,number,count
16,17,26860
26,27,26593
33,34,26412
20,21,25988
30,31,25817
23,24,25709


In [22]:
lotto_common_numbers = get_most_common_numbers(lotto)
print(lotto_common_numbers)
# lotto_plus_common_numbers = get_most_common_numbers(lotto_plus)
# print(lotto_common_numbers)

    number  count
16      17  26860
26      27  26593
33      34  26412
20      21  25988
30      31  25817
23      24  25709
    number  count
20      21   9943
29      30   9834
16      17   9809
5        6   9628
27      28   9614
6        7   9546


In [23]:
mini_lotto_common_numbers = get_most_common_numbers(mini_lotto)
print(mini_lotto_common_numbers)

    number  count
20      21   9943
29      30   9834
16      17   9809
5        6   9628
27      28   9614
6        7   9546
